<a href="https://colab.research.google.com/github/meshalalsultan/Full_Vs_Flat_Cnn_With_TensorFlow/blob/main/Tire_Fall_With_TensorFlow_with_tflite_convert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Full vs Flat Tire Images 

## Classify images of properly Filled and Underfilled Tires

The Contest and information about the data will be found here :

https://github.com/meshalalsultan/Full_Vs_Flat_Cnn_With_TensorFlow/blob/main/README.md

I get the data throw kaggle , and split them in respect to eatch class in floder name tire_dataset.zip , witch can found in https://github.com/meshalalsultan/Full_Vs_Flat_Cnn_With_TensorFlow/blob/main/tire_dataset.zip

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
#/content/drive/MyDrive/tire_dataset.zip

In [ ]:
import zipfile

# Unzip the file
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/tire_dataset.zip", "r")
zip_ref.extractall()
zip_ref.close()

In [ ]:
ls tire_dataset

In [ ]:
ls tire_dataset/train

In [ ]:
ls tire_dataset/train/full

In [ ]:
import os

# Walk through pizza_steak directory and list number of files
for dirpath, dirnames, filenames in os.walk("tire_dataset"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

## How many image in eatch class file 

In [ ]:
# Another way to find out how many images are in a file
num_full_images_train = len(os.listdir("tire_dataset/train/full"))

num_full_images_train

In [ ]:
num_flat_images_train = len(os.listdir("tire_dataset/train/flat"))

num_flat_images_train

In [ ]:
num_no_tire_images_train = len(os.listdir("tire_dataset/train/no_tire"))

num_no_tire_images_train

## Get the class name

In [ ]:
# Get the class names (programmatically, this is much more helpful with a longer list of classes)
import pathlib
import numpy as np
data_dir = pathlib.Path("tire_dataset/train/") # turn our training path into a Python path
class_names = np.array(sorted([item.name for item in data_dir.glob('*')])) # created a list of class_names from the subdirectories
print(class_names)

So now i have 261 image in train for eatch class , and 40 test 

## View the image and shape function

In [ ]:
# View an image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

def view_random_image(target_dir, target_class):
  # Setup target directory (we'll view images from here)
  target_folder = target_dir+target_class

  # Get a random image path
  random_image = random.sample(os.listdir(target_folder), 1)

  # Read in the image and plot it using matplotlib
  img = mpimg.imread(target_folder + "/" + random_image[0])
  plt.imshow(img)
  plt.title(target_class)
  plt.axis("off");

  print(f"Image shape: {img.shape}") # show the shape of the image

  return img

In [ ]:
# View a random image from the training dataset
img = view_random_image(target_dir="tire_dataset/train/",
                        target_class="full")

# Vesualize the data

In [ ]:
import matplotlib.pyplot as plt
import cv2

In [ ]:
full_img = cv2.imread('/content/tire_dataset/train/full/00000.jpg')
full_img = cv2.cvtColor(full_img,cv2.COLOR_BGR2RGB)

In [ ]:
#Check the type
type(full_img)

In [ ]:
full_img.shape

In [ ]:
plt.imshow(full_img);

Check the flat image

In [ ]:
flat_img = cv2.imread('/content/tire_dataset/train/flat/00004.jpg')
flat_img = cv2.cvtColor(flat_img,cv2.COLOR_BGRA2RGB)

In [ ]:
#Check the type
type(full_img)

In [ ]:
full_img.shape

In [ ]:
plt.imshow(full_img);

#Preprosess the image and generate more data 

scale the image from 0 , 1
by deviding /255

In [ ]:
#Test the scale
img/255

## Zoom , Scale , rotate ,flip , rescale

In [ ]:
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
img_gen = ImageDataGenerator( rotation_range=30 , #rotate the image by 30%
                             width_shift_range=0.1, #shift the img width by max 10%
                             height_shift_range=0.1, #shift the img heigh bay max 10%
                             rescale=1/255, #rescale img by normalize it
                             shear_range=0.2, #cutting away part of the img as buffer 
                             zoom_range=0.2, #zoom in be 20%
                             horizontal_flip=True,
                             fill_mode='nearest')

In [ ]:
plt.imshow(img_gen.random_transform(full_img));

In [ ]:
plt.imshow(img_gen.random_transform(full_img))

In [ ]:
plt.imshow(img_gen.random_transform(flat_img))

In [ ]:
plt.imshow(img_gen.random_transform(flat_img));

# Building the model 

## Model_1

build the model without add target_size=(240,240) in flow_from function

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#set random seed
tf.random.set_seed(42)

#Preprosses the data ( get them all in scale)
train_datagen = ImageDataGenerator( rotation_range=30 , #rotate the image by 30%
                             width_shift_range=0.1, #shift the img width by max 10%
                             height_shift_range=0.1, #shift the img heigh bay max 10%
                             rescale=1/255, #rescale img by normalize it
                             shear_range=0.2, #cutting away part of the img as buffer 
                             zoom_range=0.2, #zoom in be 20%
                             horizontal_flip=True,
                             fill_mode='nearest')

valied_datagen = ImageDataGenerator( rotation_range=30 , #rotate the image by 30%
                             width_shift_range=0.1, #shift the img width by max 10%
                             height_shift_range=0.1, #shift the img heigh bay max 10%
                             rescale=1/255, #rescale img by normalize it
                             shear_range=0.2, #cutting away part of the img as buffer 
                             zoom_range=0.2, #zoom in be 20%
                             horizontal_flip=True,
                             fill_mode='nearest')

#Setup the train and test directories
train_dir = 'tire_dataset/train/'
test_dir = 'tire_dataset/test/'

#Import data from directories and turn it into batches
train_data = train_datagen.flow_from_directory(train_dir,
                                               batch_size=32,
                                               target_size=(240,240),
                                               class_mode='categorical',
                                               seed=42)

valid_data = valied_datagen.flow_from_directory(test_dir,
                                                batch_size=32,
                                                target_size=(240,240),
                                                class_mode='categorical',
                                                seed=42)


#Create CNN Models
model_1 = tf.keras.Sequential([
  tf.keras.layers.Conv2D(filters=10,
                        kernel_size=3,
                        activation='relu',
                        input_shape=(240,240,3)),
  tf.keras.layers.Conv2D(10,3,activation='relu'),
  tf.keras.layers.MaxPool2D(pool_size=2,padding='valid'),
  tf.keras.layers.Conv2D(10,3,activation='relu'),
  tf.keras.layers.Conv2D(10,3,activation='relu'),
  tf.keras.layers.MaxPool2D(2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(3, activation='softmax') #3 class output
])

#compile Model_1
model_1.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

#Fit Model_1
history_1 = model_1.fit(train_data,
                      epochs=5,
                      steps_per_epoch=len(train_data),
                      validation_data=valid_data,
                      validation_steps=len(valid_data))


It's good .!
Model_1 did greate job with losses = 0.45 , and accuracy of the model = 0.75

The model got ~75% accouracy on the training set , and 77% on the test set.


## Check Model_1 

In [ ]:
model_1.summary()

## Model_2

run the same model but i will rehape the image size to be (224,224,3)

In [ ]:
# Set the seed 
tf.random.set_seed(42)

#Import data from directories and turn it into batches
train_data = train_datagen.flow_from_directory(train_dir,
                                               batch_size=32,
                                               target_size=(224,224),
                                               class_mode='categorical',
                                               seed=42)

valid_data = valied_datagen.flow_from_directory(test_dir,
                                                batch_size=32,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                seed=42)

#Create the model with tensorflow playground model
model_2 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(224,224,3)),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(4,activation='relu'),
    tf.keras.layers.Dense(3,activation='softmax')
])

#Compile Model_2
model_2.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

#Fit Model_2
model_2.fit(train_data,
            epochs=5,
            steps_per_epoch=len(train_data),
            validation_data=valid_data,
            validation_steps=len(valid_data))

Note Work ..!

The Image shape change it make the model bad on training with 5 epochs , i will try to increes the epoch to = 20, so that the model can take more chance to learn

## Model_3

In [ ]:
#Set Seed
tf.random.set_seed(42)

#Create the model
model_3 = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(224,224,3)),
  tf.keras.layers.Dense(4, activation='relu'),
  tf.keras.layers.Dense(4, activation='relu'),
  tf.keras.layers.Dense(3,activation='softmax')
])

#Compile Model_3
model_3.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

#Fit Model_3
history_3 = model_3.fit(train_data,
                        epochs=20,
                        steps_per_epoch=len(train_data),
                        validation_data=valid_data,
                        validation_steps=len(valid_data))

So , It seems our model not learning anything .!

for model_4 i will try to give the model more nuron to train with 4 > 100, and add extra layers 

## Model_4

In [ ]:
#Set random Seed
tf.random.set_seed(42)

#Greate Model_4
model_4 = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(224,224,3)),
  tf.keras.layers.Dense(100 , activation='relu'),
  tf.keras.layers.Dense(100 , activation='relu'),
  tf.keras.layers.Dense(100 ,activation='relu'),
  tf.keras.layers.Dense(3, activation='softmax')
])

#Compile Model_4
model_4.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

#Fit Model_4
history_4 = model_4.fit(train_data,
                        epochs=20,
                        steps_per_epoch=len(train_data),
                        validation_data=valid_data,
                        validation_steps=len(valid_data))

Good job Model_4 , It's seems model_4 get grate job and learn some pattren in the data , it give 72% accouracy in training data , 72% accouracy in test data.

So , To tune the model litteil more , the 72% accourcy it's seems not vary good to me :(


**BUT**
Start from `model_5` , i will get back to Conv2d Layers , as i make `model_1` as basline model

and i will start tune the model from there.

## Model_5

I will start same `model_1` with more epochs .

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Activation

In [ ]:
#Set random seed
tf.random.set_seed(42)

#Create the model
model_5= Sequential([
  tf.keras.layers.Conv2D(
      filters=10,
      kernel_size=3,
      activation='relu',
      input_shape=(224,224,3)),
  tf.keras.layers.Conv2D(10,3,activation='relu'),
  tf.keras.layers.MaxPool2D(pool_size=2,padding='valid'),
  tf.keras.layers.Conv2D(10,3,activation='relu'),
  tf.keras.layers.Conv2D(10,3,activation='relu'),
  tf.keras.layers.MaxPool2D(2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(3,activation='softmax')
])

#Compile Model_5
model_5.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

#Fit Model_5
history_5 = model_5.fit(train_data,
                      epochs=5,
                      steps_per_epoch=len(train_data),
                      validation_data=valid_data,
                      validation_steps=len(valid_data))

## Model_6 

Start from `model_5` as basline i will :

- Adjast the code 
- But the augamented data witch i create in strter of this notebook
- add more neuron 

In [ ]:
#Create Augmented data
train_data_aug = ImageDataGenerator( rotation_range=30 , #rotate the image by 30%
                             width_shift_range=0.1, #shift the img width by max 10%
                             height_shift_range=0.1, #shift the img heigh bay max 10%
                             rescale=1/255, #rescale img by normalize it
                             shear_range=0.2, #cutting away part of the img as buffer 
                             zoom_range=0.2, #zoom in be 20%
                             horizontal_flip=True,
                             fill_mode='nearest')

In [ ]:
# Import data and augment it from training directory
print("Augmented training images:")
train_data_augmented = train_data_aug.flow_from_directory(train_dir,
                                                                   target_size=(224, 224),
                                                                   batch_size=32,
                                                                   class_mode='categorical',
                                                                   shuffle=False) # Don't shuffle for demonstration purposes, usually a good thing to shuffle


In [ ]:
# Get data batch samples
images, labels = train_data.next()
augmented_images, augmented_labels = train_data_augmented.next() # Note: labels aren't augmented, they stay the same

In [ ]:
# Create ImageDataGenerator training instance without data augmentation
train_datagen = ImageDataGenerator(rescale=1/255.) 

# Create ImageDataGenerator test instance without data augmentation
test_datagen = ImageDataGenerator(rescale=1/255.)

In [ ]:
# Create non-augmented data batches
print("Non-augmented training images:")
train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=(224, 224),
                                               batch_size=32,
                                               class_mode='categorical',
                                               shuffle=True) # Don't shuffle for demonstration purposes

print("Unchanged test images:")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=(224, 224),
                                             batch_size=32,
                                             class_mode='categorical')

In [ ]:
# Create the model (this can be our baseline, a 3 layer Convolutional Neural Network)
model_6 = Sequential([
  Conv2D(10, 3, activation='relu', input_shape=(224, 224, 3)),
  MaxPool2D(pool_size=2), # reduce number of features by half
  Conv2D(10, 3, activation='relu'),
  MaxPool2D(),
  Conv2D(10, 3, activation='relu'),
  MaxPool2D(),
  Flatten(),
  Dense(1, activation='sigmoid')
])

# Compile Model_6 
model_6.compile(loss='binary_crossentropy',
                optimizer=Adam(),
                metrics=['accuracy'])

#Fit Model_6
history_6 = model_6.fit(train_data_augmented,
                      epochs=5,
                      steps_per_epoch=len(train_data_augmented),
                      validation_data=test_data,
                      validation_steps=len(test_data))

In [ ]:
model_6.summary()

### Plot Function

In [ ]:
# Plot the validation and training data separately
def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.
  """ 
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

In [ ]:
plot_loss_curves(history_6)

I littel wory about overfitting , Becouse the curv of accouracy , i will try to fix it now 

## Model_7

Make the train data shuffeld

In [ ]:
print("Augmented training images:")
train_data_shuffld = train_data_aug.flow_from_directory(train_dir,
                                                                   target_size=(224, 224),
                                                                   batch_size=32,
                                                                   class_mode='categorical',
                                                                   shuffle=True)

In [ ]:
# Set the seed 
tf.random.set_seed(42)

#Create Model_7
model_7 = Sequential([
  Conv2D(10,3,activation='relu', input_shape=(224,224,3)),
  MaxPool2D(),
  Conv2D(10,3,activation='relu'),
  MaxPool2D(),
  Conv2D(10,3,activation='relu'),
  MaxPool2D(),
  Flatten(),
  Dense(3,activation='softmax')
])

#Compile Model_7
model_7.compile(loss='categorical_crossentropy',
                optimizer=Adam(),
                metrics=['accuracy']
                )

#Fit the model
history_7 = model_7.fit(train_data_shuffld,
                        epochs=5,
                        steps_per_epoch=len(train_data_shuffld),
                        validation_data=test_data,
                        validation_steps=len(test_data))

In [ ]:
plot_loss_curves(history_7)

Its Learning good , So I will Add more epoch AKA:More chance to learn

## Model_8

In [ ]:
#Set Seed
tf.random.set_seed(42)

#Create Model_8
model_8 = Sequential([
  Conv2D(10,3,activation='relu',input_shape=(224,224,3)),
  MaxPool2D(),
  Conv2D(10,3,activation='relu'),
  MaxPool2D(),
  Conv2D(10,3,activation='relu'),
  MaxPool2D(),
  Flatten(),
  Dense(3,activation='softmax')
  ])

#Compile Model_8
model_8.compile(loss='categorical_crossentropy',
                optimizer=Adam(),
                metrics=['accuracy'])

#Fit Model_8
history_8 = model_8.fit(train_data_shuffld,
                        epochs=20,
                        steps_per_epoch=len(train_data_shuffld),
                        validation_data=test_data,
                        validation_steps=len(test_data))

In [ ]:
plot_loss_curves(history_8)

Here are fairly Good model :)

The model got 82% accuracy ,
Let me fine tune the neroune or maybe add 1 more layer

## Model_9

In [ ]:
#Set Seed
tf.random.set_seed(42)

#Create Model_9
model_9 = Sequential([
  Conv2D(10,3,activation='relu',input_shape=(224,224,3)),
  MaxPool2D(),
  Conv2D(10,3,activation='relu'),
  MaxPool2D(),
  Conv2D(10,3,activation='relu'),
  MaxPool2D(),
  Conv2D(10,3,activation='relu'),
  MaxPool2D(),
  Flatten(),
  Dense(3,activation='softmax')
  ])

#Compile Model_9
model_9.compile(loss='categorical_crossentropy',
                optimizer=Adam(),
                metrics=['accuracy'])

#Fit Model_9
history_9 = model_9.fit(train_data_shuffld,epochs=20,steps_per_epoch=len(train_data_shuffld),validation_data=test_data,validation_steps=len(test_data))

In [ ]:
plot_loss_curves(history_9)

# It's Officaly 90% Accouracy :)



Give the Model more epochs to learn , 20 > 30

## Model_10

In [ ]:
#Set Seed
tf.random.set_seed(42)

#Clone Model_9 -> Model_10
model_10 = tf.keras.models.clone_model(model_9)

#Compile Model_10
model_10.compile(loss='categorical_crossentropy',
                 optimizer=Adam(),
                 metrics=['accuracy'])

#Fit Model_10
history_10 = model_10.fit(train_data_shuffld,epochs=30,steps_per_epoch=len(train_data_shuffld),validation_data=test_data,validation_steps=len(test_data))

# It's 93% Accuracy :)

In [ ]:
plot_loss_curves(history_10)

In [ ]:
model_10.evaluate(test_data)

Even thow , The model_10 is overfitting in test_data.

Let's try make predection on new image

In [ ]:
# Create a function to import an image and resize it to be able to be used with our model
def load_and_prep_image(filename, img_shape=224):
  """
  Reads an image from filename, turns it into a tensor
  and reshapes it to (img_shape, img_shape, colour_channel).
  """
  # Read in target file (an image)
  img = tf.io.read_file(filename)

  # Decode the read file into a tensor & ensure 3 colour channels 
  # (our model is trained on images with 3 colour channels and sometimes images have 4 colour channels)
  img = tf.image.decode_image(img, channels=3)

  # Resize the image (to the same size our model was trained on)
  img = tf.image.resize(img, size = [img_shape, img_shape])

  # Rescale the image (get all values between 0 and 1)
  img = img/255.
  return img

In [ ]:
#Create function to scale and get the image in shape of the model
def pred_and_plot(model, filename, class_names):
  """
  Imports an image located at filename, makes a prediction on it with
  a trained model and plots the image with the predicted class as the title.
  """
  # Import the target image and preprocess it
  img = load_and_prep_image(filename)

  # Make a prediction
  pred = model.predict(tf.expand_dims(img, axis=0))

  # Get the predicted class
  pred_class = class_names[int(tf.round(pred)[0][0])]

  # Plot the image and predicted class
  plt.imshow(img)
  plt.title(f"Prediction: {pred_class}")
  plt.axis(False);

In [ ]:
class_names

# Make Predection

In [ ]:
# Make a prediction using model_10
pred_and_plot(model=model_10, 
              filename="tire0.jpeg", 
              class_names=class_names)

In [ ]:
pred_and_plot(model=model_10, 
              filename="tire02.jpeg", 
              class_names=class_names)

In [ ]:
pred_and_plot(model=model_10, 
              filename="tire03.jpeg", 
              class_names=class_names)

In [ ]:
pred_and_plot(model=model_10, 
              filename="tire04.jpeg", 
              class_names=class_names)

In [ ]:
pred_and_plot(model=model_10, 
              filename="tire05.jpeg", 
              class_names=class_names)

In [ ]:
pred_and_plot(model=model_10, 
              filename="tire1.jpeg", 
              class_names=class_names)

#Get the Model in tflite Format Label and Model for flutter



In [ ]:
print (train_data_shuffld.class_indices)

labels = '\n'.join(sorted(train_data_shuffld.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_10)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)